# SiGesGuarda Database

https://www.curitiba.pr.gov.br/dadosabertos/busca/?termo=guarda

In this notebook, I do some preprocessing for the database:
- standartization of features 
- drop of non interesting features
- cleaning bad data
- translation of feature names
- creation of a 'category' feature
- filtering of the database (for now, I'm only interested in the category 'criminal behaviour')
- translation of the itens in the category criminal behaviour

The final database is saved under db_criminalBehaviour.csv

Importing packages

In [2]:
import pandas as pd
import json
import unicodedata
import numpy as np


Exploring the database characteristics

In [60]:
df = pd.read_csv('./2023-07-01_sigesguarda_-_Base_de_Dados.txt', sep="\t", encoding='latin-1')


C:\Users\vanes\AppData\Local\Temp\ipykernel_1156\1921366003.py:1: DtypeWarning: Columns (0,6,8,10,12,14,15,19,20,21,22,26,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./2023-07-01_sigesguarda_-_Base_de_Dados.txt', sep="\t", encoding='latin-1')


In [61]:
df.drop_duplicates(inplace=True)
df.drop([0], axis=0, inplace=True) # drop the line with dashes

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429346 entries, 1 to 429346
Data columns (total 35 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   ATENDIMENTO_ANO            429327 non-null  object
 1   ATENDIMENTO_BAIRRO_NOME    429201 non-null  object
 2   EQUIPAMENTO_URBANO_NOME    152086 non-null  object
 3   FLAG_EQUIPAMENTO_URBANO    429346 non-null  object
 4   FLAG_FLAGRANTE             429346 non-null  object
 5   LOGRADOURO_NOME            429346 non-null  object
 6   NATUREZA1_DEFESA_CIVIL     429346 non-null  object
 7   NATUREZA1_DESCRICAO        429346 non-null  object
 8   NATUREZA2_DEFESA_CIVIL     22174 non-null   object
 9   NATUREZA2_DESCRICAO        22174 non-null   object
 10  NATUREZA3_DEFESA_CIVIL     1668 non-null    object
 11  NATUREZA3_DESCRICAO        1668 non-null    object
 12  NATUREZA4_DEFESA_CIVIL     288 non-null     object
 13  NATUREZA4_DESCRICAO        288 non-null     

In [42]:
for col in df.columns:
    print("Column: ", col, "\n", df[col].unique())

Column:  ATENDIMENTO_ANO 
 ['2009' 2009 2010 2011 2012 2013 nan 2014.0 2015.0 2016 2017.0 2018 2019
 2020 2021.0 2022]
Column:  ATENDIMENTO_BAIRRO_NOME 
 ['CIDADE INDUSTRIAL' 'FAZENDINHA' 'UBERABA' 'SÍTIO CERCADO' 'TATUQUARA'
 'SANTA CÂNDIDA' 'BOQUEIRÃO' 'CENTRO' 'BOA VISTA' 'TABOÃO' 'XAXIM'
 'PILARZINHO' 'REBOUÇAS' 'ÁGUA VERDE' 'BATEL' 'NOVO MUNDO'
 'ALTO BOQUEIRÃO' 'CAPÃO RASO' 'JARDIM BOTÂNICO' 'PORTÃO' 'ORLEANS'
 'SANTA FELICIDADE' 'CASCATINHA' 'CAPÃO DA IMBUIA' 'BARREIRINHA'
 'SEMINÁRIO' 'CAMPO COMPRIDO' 'PRADO VELHO' 'PINHEIRINHO' 'BUTIATUVINHA'
 'CAMPINA DO SIQUEIRA' 'CAJURU' 'SÃO FRANCISCO' 'CENTRO CÍVICO' 'SÃO BRAZ'
 'UMBARÁ' 'CAXIMBA' 'JARDIM SOCIAL' 'BACACHERI' 'CAMPO DE SANTANA'
 'SANTO INÁCIO' 'JARDIM DAS AMÉRICAS' 'LINDÓIA' 'GANCHINHO' 'PAROLIN'
 'ABRANCHES' 'SÃO JOÃO' 'ATUBA' 'TARUMÃ' 'ALTO DA RUA XV' 'MOSSUNGUÊ'
 'TINGUI' 'BIGORRILHO' 'BAIRRO ALTO' 'HAUER' 'VILA IZABEL' 'CABRAL'
 'BOM RETIRO' 'GUAÍRA' 'CACHOEIRA' 'AUGUSTA' 'CRISTO REI' 'AHÚ'
 'ALTO DA GLÓRIA' 'GUABIROTU

In [43]:
df.columns

Index(['ATENDIMENTO_ANO', 'ATENDIMENTO_BAIRRO_NOME', 'EQUIPAMENTO_URBANO_NOME',
       'FLAG_EQUIPAMENTO_URBANO', 'FLAG_FLAGRANTE', 'LOGRADOURO_NOME',
       'NATUREZA1_DEFESA_CIVIL', 'NATUREZA1_DESCRICAO',
       'NATUREZA2_DEFESA_CIVIL', 'NATUREZA2_DESCRICAO',
       'NATUREZA3_DEFESA_CIVIL', 'NATUREZA3_DESCRICAO',
       'NATUREZA4_DEFESA_CIVIL', 'NATUREZA4_DESCRICAO',
       'NATUREZA5_DEFESA_CIVIL', 'NATUREZA5_DESCRICAO',
       'SUBCATEGORIA1_DESCRICAO', 'SUBCATEGORIA2_DESCRICAO',
       'SUBCATEGORIA3_DESCRICAO', 'SUBCATEGORIA4_DESCRICAO',
       'SUBCATEGORIA5_DESCRICAO', 'OCORRENCIA_ANO', 'OCORRENCIA_CODIGO',
       'OCORRENCIA_DATA', 'OCORRENCIA_DIA_SEMANA', 'OCORRENCIA_HORA',
       'OCORRENCIA_MES', 'OPERACAO_DESCRICAO', 'ORIGEM_CHAMADO_DESCRICAO',
       'REGIONAL_FATO_NOME', 'SECRETARIA_NOME', 'SECRETARIA_SIGLA',
       'SERVICO_NOME', 'SITUACAO_EQUIPE_DESCRICAO', 'NUMERO_PROTOCOLO_156'],
      dtype='object')

Normalizing features, deleting unnecessary columns

In [62]:
columns2drop = ['ATENDIMENTO_ANO','OCORRENCIA_CODIGO']

strFeatures = [ 
                'ATENDIMENTO_BAIRRO_NOME', 'EQUIPAMENTO_URBANO_NOME', 'LOGRADOURO_NOME', 
                'NATUREZA1_DESCRICAO', 'NATUREZA2_DESCRICAO','NATUREZA3_DESCRICAO','NATUREZA4_DESCRICAO','NATUREZA5_DESCRICAO',
                'SUBCATEGORIA1_DESCRICAO','SUBCATEGORIA2_DESCRICAO','SUBCATEGORIA3_DESCRICAO','SUBCATEGORIA4_DESCRICAO','SUBCATEGORIA5_DESCRICAO',
                'OCORRENCIA_DIA_SEMANA','OPERACAO_DESCRICAO', 'ORIGEM_CHAMADO_DESCRICAO',
                'REGIONAL_FATO_NOME', 'SECRETARIA_NOME', 'SECRETARIA_SIGLA',
                'SERVICO_NOME', 'SITUACAO_EQUIPE_DESCRICAO',
               ]

boolFeatures = ['NATUREZA1_DEFESA_CIVIL', 'NATUREZA2_DEFESA_CIVIL', 'NATUREZA3_DEFESA_CIVIL', 'NATUREZA4_DEFESA_CIVIL', 'NATUREZA5_DEFESA_CIVIL']

numFeatures = ['NUMERO_PROTOCOLO_156']

dateFeatures = ['OCORRENCIA_DATA']

In [63]:
df.drop(columns2drop, axis=1, inplace=True)
df[boolFeatures] = df[boolFeatures].apply(pd.to_numeric)
df[numFeatures] = df[numFeatures].apply(pd.to_numeric)
df[dateFeatures] = df[dateFeatures].apply(pd.to_datetime)

for feat in strFeatures:
    df[feat] = df[feat].str.normalize('NFKD')

In [64]:
df['OCORRENCIA_ANO']  = df['OCORRENCIA_DATA'].dt.year 
df['OCORRENCIA_MES']  = df['OCORRENCIA_DATA'].dt.month 
df['OCORRENCIA_DIA']  = df['OCORRENCIA_DATA'].dt.day 
df['OCORRENCIA_HORA'] = df['OCORRENCIA_DATA'].dt.hour 

In [65]:
weekDay = {
    'DOMINGO': '1-Sunday',
    'SEGUNDA': '2-Monday',
    'TERÇA'  : '3-Tuesday',
    'QUARTA' : '4-Wednesday',
    'QUINTA' : '5-Thursday',
    'SEXTA'  : '6-Friday',
    'SÁBADO' : '7-Saturday',
}
df['OCORRENCIA_DIA_SEMANA'] = [weekDay[x] for x in df['OCORRENCIA_DIA_SEMANA']]


Verifying the neighbourhood and address columns

In [66]:
neighbourhood_df = df.sort_values(by=["ATENDIMENTO_BAIRRO_NOME"], ascending=True)['ATENDIMENTO_BAIRRO_NOME'].unique()
neighbourhood_df

array([' JARDIM OSASCO', ' JARDIM PEDRO DEMETE', 'ABRANCHES',
       'AFONSO PENA', 'AFONSO PENA ', 'AGUAS BELAS', 'AHÚ',
       'ALTO BOQUEIRÃO', 'ALTO DA GLÓRIA', 'ALTO DA RUA XV', 'ATUBA',
       'AUGUSTA', 'ÁGUA VERDE', 'ÁGUAS BELAS', 'BACACHERI',
       'BAIRRO ALTO', 'BAIRRO FICTÍCIO', 'BAIRRO NAO INFORMADO',
       'BAIRRO NÃO LOCALIZAD', 'BARIGUI', 'BARREIRINHA', 'BARRO PRETO',
       'BATEL', 'BELAS AGUAS', 'BIGORRILHO', 'BOA VISTA', 'BOM RETIRO',
       'BOQUEIRÃO', 'BOQUEIRÃO ', 'BORDA DO CAMPO', 'BORDA DO CAMPO ',
       'BRAGA', 'BUTIATUVINHA', 'CABRAL', 'CACHOEIRA', 'CAJURU',
       'CAMPINA DO SIQUEIRA', 'CAMPINHA GRANDE DO S', 'CAMPO COMPRIDO',
       'CAMPO DE SANTANA', 'CAMPO DE SÃO BENEDIT', 'CAMPO LARGO',
       'CAMPO PEQUENO', 'CAMPO PEQUENO ', 'CANGUIRI', 'CAPÃO DA IMBUIA',
       'CAPÃO RASO', 'CASCATINHA', 'CAXIMBA', 'CENTRO', 'CENTRO ',
       'CENTRO CÍVICO', 'CIC', 'CIDADE INDUSTRIAL',
       'CIDADE INDUSTRIAL DE', 'CIDADE JARDIM', 'COLOMBO',
 

In [68]:
address_df = df.sort_values(by=["LOGRADOURO_NOME"], ascending=True)['LOGRADOURO_NOME'].unique()
print([x for x in address_df if 'NÃO' in x])

[' LOGRADOURO NÃO DENOMINADO', 'FERNÃO DIAS PAIS', 'LOGRADOURO NÃO DENOMINADO', 'NÃO EXISTE', 'NÃO INFORMADA', 'NÃO INFORMADO', 'RUA NÃO CADASTRADA']


In [69]:
neighbourhood_nan = ['BAIRRO FICTÍCIO', 'BAIRRO NAO INFORMADO','BAIRRO NÃO LOCALIZAD','INDICAÇÕES CANCELADA','NÃO ENCONTRADO',
       'NÃO INFORMADO ', 'NF', 'NI']
df.loc[:,'ATENDIMENTO_BAIRRO_NOME'] = [np.nan if x in neighbourhood_nan else x for x in df['ATENDIMENTO_BAIRRO_NOME']]

address_nan = [' LOGRADOURO NÃO DENOMINADO', 'LOGRADOURO NÃO DENOMINADO', 'NÃO EXISTE', 'NÃO INFORMADA', 'NÃO INFORMADO', 'RUA NÃO CADASTRADA']
df.loc[:,'LOGRADOURO_NOME'] = [np.nan if x in address_nan else x for x in df['LOGRADOURO_NOME']]


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429346 entries, 1 to 429346
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   ATENDIMENTO_BAIRRO_NOME    429168 non-null  object        
 1   EQUIPAMENTO_URBANO_NOME    152086 non-null  object        
 2   FLAG_EQUIPAMENTO_URBANO    429346 non-null  object        
 3   FLAG_FLAGRANTE             429346 non-null  object        
 4   LOGRADOURO_NOME            429224 non-null  object        
 5   NATUREZA1_DEFESA_CIVIL     429346 non-null  int64         
 6   NATUREZA1_DESCRICAO        429346 non-null  object        
 7   NATUREZA2_DEFESA_CIVIL     22174 non-null   float64       
 8   NATUREZA2_DESCRICAO        22174 non-null   object        
 9   NATUREZA3_DEFESA_CIVIL     1668 non-null    float64       
 10  NATUREZA3_DESCRICAO        1668 non-null    object        
 11  NATUREZA4_DEFESA_CIVIL     288 non-null     float64 

Translating the feature names:

In [71]:
# read json, transform in dictionary, rename columns
featureNames = pd.read_json('features_translation.json', orient='index')
featureNames.reset_index(inplace=True)
featureNames.columns = ['portuguese', 'english']
featureNames = {x: y for x,y in zip(featureNames['portuguese'],featureNames['english'])}
df.columns = [featureNames[x] for x in df.columns]


For now, I'm only interested in criminal behaviour, so I created a json file with categories to filter the database. I also created a json for the translation of this category. After filtering and translating the category items, I save the dataframe into a new csv.

In [72]:
with open('./categories.json') as f:
	json_s = f.read()
	categories = json.loads(json_s)

keys = [unicodedata.normalize('NFKD', x) for x in categories.keys()]
categories = dict(zip(keys,categories.values()))
df['categories'] = [categories[x] for x in df['problem_type_1']]

In [73]:
with open('./category_criminal_translation.json') as f:
	json_s = f.read()
	problemType1_translation = json.loads(json_s)
keys = [unicodedata.normalize('NFKD', x) for x in problemType1_translation.keys()]
problemType1_translation = dict(zip(keys,problemType1_translation.values()))


In [74]:
dfCrim = df[df['categories']=='CriminalBehaviour']
dfCrim.loc[:,'problem_type_1'] = [problemType1_translation[x] for x in dfCrim['problem_type_1']]

In [75]:
dfCrim.to_csv('./db_criminalBehaviour.csv', sep='\t')


The next steps are done in a second notebook